In [1]:
# change to tf 2.0 (run on colab)
# !pip uninstall tensorflow
# !pip install tensorflow==2.0.0

In [ ]:
# Load git code
!git clone -s https://VincentHenric:@github.com/VincentHenric/Deep_learning_project.git
%cd Deep_learning_project
!ls

from google.colab import files

In [ ]:
# To use dynamic batches
!pip install pybind11
!./compile.sh

In [ ]:
# Access Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Move data to working folder
%mv -v "../content/gdrive/My Drive/deep_learning_project/*" "/content/Deep_learning_project/"

In [ ]:
%load_ext autoreload
%autoreload 1

import numpy as np
from matplotlib import pyplot as plt

import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tesnroflow.keras.callbacks import ModelCheckpoint
import golois
print(tf.__version__)

In [ ]:
PLANES = 8
MOVES = 361
DYNAMIC_BATCH = False
model_name = ""

In [ ]:
if DYNAMIC_BATCH:
    N = 100000
    input_data = np.random.randint(2, size=(N, 19, 19, PLANES))
    input_data = input_data.astype ('float32')
    
    policy = np.random.randint(MOVES, size=(N,))
    policy = keras.utils.to_categorical (policy)
    
    value = np.random.randint(2, size=(N,))
    value = value.astype ('float32')
    
    end = np.random.randint(2, size=(N, 19, 19, 2))
    end = end.astype ('float32')

    golois.getBatch (input_data, policy, value, end)
else:
    input_data = np.load ('input_data.npy')
    policy = np.load ('policy.npy')
    value = np.load ('value.npy')
    end = np.load ('end.npy')

models = os.listdir('/models')
models = [file[:-3] for file in models if file.endswith('.h5')]
if model_name in models:
    model = model.load('{}.h5'.format(model_name))
else:
    input = keras.Input(shape=(19, 19, PLANES), name='board')
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(input)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    policy_head = layers.Conv2D(1, 3, activation='relu', padding='same')(x)
    policy_head = layers.Flatten()(policy_head)
    policy_head = layers.Dense(MOVES, activation='softmax', name='policy')(policy_head)
    value_head = layers.Flatten()(x)
    value_head = layers.Dense(1, activation='sigmoid', name='value')(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])

model.summary ()

model.compile(optimizer=keras.optimizers.SGD(lr=0.1),
              loss={'value': 'mse', 'policy': 'categorical_crossentropy'})

In [ ]:
checkpointer = ModelCheckpoint(filepath='models/{}.h5'.format(model_name),
                                   verbose=1,
                                   save_best_only=True)

print('Start learning')
model.fit(input_data, {'policy': policy, 'value': value},
          epochs=20, batch_size=128, validation_split=0.1,
          callbacks=[checkpointer])